Looking at this dataframe, I notice there are a lot of `sparse` (null-filled) rows. In addition the `description_tokens` it's unhelpful in displaying what skills are required for a job. 

I'll clean up the dataset by doing the following below.

Read csv, assign a variable to it

In [87]:
import pandas as pd
import csv

# Print first 5 rows
job_df = pd.read_csv("data/joined_data.csv")
job_df.head()


,id,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,title,company_name,location,via,extensions,posted_at,schedule_type,work_from_home,date_time,description_tokens
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Analyst Technical - Consultant,Intermountain Healthcare,"Oklahoma City, OK",via Monster,"['19 hours ago', 'Full-time', 'Health insurance']",19 hours ago,Full-time,NaN,00:10.1,"['vba', 'spreadsheet', 'javascript', 'word', '..."
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sr Analyst, Data Engineer",Estee Lauder Companies,Oklahoma,via ZipRecruiter,"['12 hours ago', 'Full-time']",12 hours ago,Full-time,NaN,00:10.1,[]
2,2,21 to 23,an hour,22.0,21.0,23.0,22.0,NaN,45760.0,Legal Data Analyst,DISYS,United States,via Indeed,"['4 hours ago', '21 to 23 an hour', 'Contractor']",4 hours ago,Contractor,NaN,00:10.1,[]
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data analyst,Modis,United States,via BeBee,"['13 hours ago', 'No degree mentioned']",13 hours ago,NaN,NaN,00:10.1,[]
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Analyst (Risk Adjustment Consulting Resea...,Cambia Health Solutions,United States,via LinkedIn,"['23 hours ago', 'Full-time', 'Health insuranc...",23 hours ago,Full-time,NaN,00:10.1,"['sas', 'matlab', 'sql', 'spss', 'r']"


Explore how many missing values are in the dataset

In [88]:
# First check how many nulls there are

job_df["work_from_home"].isna().sum()

6004

In [89]:
# Column has 5 floats inside, must drop them

job_df["description_tokens"].isna().sum()

5

In [90]:
# Dropped the 5 floats

job_df.dropna(subset = "description_tokens", inplace = True)

job_df["description_tokens"].isna().sum()

0

In [91]:
job_df["work_from_home"] = job_df["work_from_home"].fillna(value = False)

Transform the description_tokens column into a column of lists

In [92]:
from sklearn.preprocessing import MultiLabelBinarizer
from ast import literal_eval

mlb = MultiLabelBinarizer()

# Convert description_tokens into col of lists

job_df["description_tokens"] = job_df["description_tokens"].apply(literal_eval)


Join the results of the MultiLabelBinarizer back to the dataframe

In [93]:
# Encode each skill as a unique column

binary_columns = mlb.fit_transform(job_df["description_tokens"].tolist())
print(mlb.classes_)

# Create new df with newly engineered features 
# Save to data folder for future use

encoded_df = job_df.join(pd.DataFrame(binary_columns, columns = mlb.classes_, index = job_df.index))
encoded_df.to_csv("data/joined_data.csv", index = False)


['airflow' 'alteryx' 'apl' 'asp.net' 'assembly' 'atlassian' 'aurora' 'aws'
 'azure' 'bash' 'bigquery' 'bitbucket' 'c' 'c++' 'c/c++' 'cobol' 'cognos'
 'crystal' 'css' 'dart' 'dax' 'docker' 'dplyr' 'excel' 'fortran' 'gcp'
 'gdpr' 'ggplot2' 'git' 'github' 'gitlab' 'go' 'golang' 'graphql' 'groovy'
 'hadoop' 'html' 'java' 'javascript' 'jira' 'jquery' 'js' 'julia'
 'jupyter' 'keras' 'linux' 'linux/unix' 'looker' 'matlab' 'matplotlib'
 'microstrategy' 'mongo' 'mongodb' 'mssql' 'mysql' 'no-sql' 'node'
 'node.js' 'nosql' 'nuix' 'numpy' 'outlook' 'pandas' 'perl' 'php' 'pl/sql'
 'plotly' 'postgres' 'postgresql' 'power_bi' 'powerpoint' 'powerpoints'
 'powershell' 'pyspark' 'python' 'pytorch' 'qlik' 'r' 'redis' 'redshift'
 'rshiny' 'ruby' 'rust' 'sap' 'sas' 'scala' 'scikit-learn' 'seaborn'
 'selenium' 'sharepoint' 'shell' 'snowflake' 'solidity' 'spark' 'splunk'
 'spreadsheet' 'spss' 'sql' 'ssis' 'ssrs' 'swift' 't-sql' 'tableau'
 'tensorflow' 'terminal' 'tidyr' 'twilio' 'typescript' 'unix' 'unix/lin

Drop all null values in encoded_df column 'salary_standardized'


In [94]:
# Verify the nulls

encoded_df.isna().sum()['salary_standardized']


8976

In [95]:
# Drop the nulls

encoded_df.dropna(subset = 'salary_standardized', inplace = True)


In [96]:
encoded_df.isna().sum()['salary_standardized']

salary_standard_clean = encoded_df

In [97]:
# Save new df into data folder

salary_standard_clean.to_csv("data/encoded_df.csv", index = False)
